In [1]:
# List all the models aavailable in Amazon Bedrock

import boto3
import pandas as pd

# Create a Bedrock client
bedrock_client = boto3.client('bedrock')

# Create a Bedrock Runtime client
bedrock_runtime_client = boto3.client('bedrock-runtime')

In [ ]:
# store all the models in a variable
all_models = bedrock_client.list_foundation_models()
all_models_summaries = all_models['modelSummaries'] 

# Convert the model summaries to a DataFrame with only the columns we want
df = pd.DataFrame(all_models_summaries, columns=['modelName', 'providerName', 'modelId', 'modelArn', 'inputModalities', 'outputModalities'])

# Show the DataFrame without truncating the output and display all rows
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# Print the models that are provided by Amazon and show the first 10 rows and select the columns modelName, providerName, modelId
df[df['providerName'] == 'Amazon'][:10][['modelName', 'providerName', 'modelId']]

In [ ]:
# Print all the embedding models
df[df['modelId'].str.contains('emb')][['modelName', 'providerName', 'modelId', 'inputModalities', 'outputModalities']]


In [ ]:
from langchain_community.document_loaders import BSHTMLLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_aws import BedrockEmbeddings
from langchain_community.vectorstores import FAISS
import os

def create_vector_store(index_name="2023WC"):
    # Create output directory if it doesn't exist
    os.makedirs("data", exist_ok=True)
    
    # Load the HTML file
    loader = BSHTMLLoader("data/2023WC.html")
    data = loader.load()
    
    # Split the text into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(data)
    
    # Initialize embeddings
    embeddings = BedrockEmbeddings(model_id='amazon.titan-embed-text-v2:0')
    
    # Create FAISS vector store
    db = FAISS.from_documents(chunks, embeddings)
    
    # Save the vector store
    db.save_local("data", index_name)
    print("Vector store created successfully!")
    
    return db
    

index = create_vector_store(index_name="2023WC")

In [77]:
import re
from langchain_aws import BedrockEmbeddings, ChatBedrock
from langchain_community.vectorstores import FAISS

In [93]:
# Function to clean and standardize text by removing unwanted elements
def clean_text(text):
    # Convert non-breaking spaces to regular spaces
    text = text.replace('\xa0', ' ')
    
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # Remove citation references like [1], [2,3], etc.
    text = re.sub(r'\[.*?\]', '', text)
    
    # Normalize whitespace by removing extra spaces and line breaks
    text = ' '.join(text.split())
    
    return text

def rag_function(query, index):
    
    # Retrieve relevant documents
    retrieved_docs = index.similarity_search(query, k=2)
    retrieved_context = [clean_text(retrieved_docs[0].page_content + retrieved_docs[1].page_content)]

    # Create augmented prompt
    augmented_prompt = f"""
    Given the context below answer the question.

    Question: {query} 

    Context: {retrieved_context}

    Remember to answer only based on the context provided and not from any other source. 

    If the question cannot be answered based on the provided context, say I don't know.
    """

    # Initialize Bedrock Claude model
    llm = ChatBedrock(
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",
        model_kwargs={
            "temperature": 0,
            "max_tokens": 2048
        }
    )

    # Generate response
    response = llm.invoke(augmented_prompt)

    return retrieved_context, response.content

In [108]:
def display_rag_results(context, answer):
    # Display the context and answer with markdown
    from IPython.display import Markdown

    md_text = f"""
    ### Context:
    {context}

    ### Answer:
    {answer}
    """

    display(Markdown(md_text))


In [ ]:
query = "What RAG?"
context, answer = rag_function(query, index)


display_rag_results(context, answer)

In [ ]:
query = "Who won the world cup?"
context, answer = rag_function(query, index)


display_rag_results(context, answer)

In [ ]:
query = "What was Virat Kohli's achievement in the Cup?"
context, answer = rag_function(query, index)


display_rag_results(context, answer)